In [ ]:
%pip install sentence-transformers

In [2]:
import sys; sys.path.append('..')

import pandas as pd

from common.polymarket import load_market_data
from common.news import MARKET_KEYWORDS, fetch_news_for_period, filter_by_relevance, save_news_to_csv

In [4]:
# Configuration
EVENT_ID = 13551
MARKET_NAME = "ukraine"
DATA_FILE = "trump-wins-ends-ukraine-war-in-90-days-2c482.json"

# Load market data
data = load_market_data(filename=DATA_FILE)
event_info = data['event']
start_date = event_info['start_date']
end_date = event_info['end_date']

# Get keywords for this market
keywords = MARKET_KEYWORDS.get((EVENT_ID, MARKET_NAME), [])

print(f"Event: {event_info['title']}")
print(f"Period: {start_date.date()} to {end_date.date()}")
print(f"Duration: {(end_date - start_date).days} days")
print(f"Keywords: {keywords}")

Loading times: 100%|██████████| 258658/258658 [00:34<00:00, 7408.16point/s]

Event: Trump ends Ukraine war in first 90 days?
Period: 2024-10-22 to 2025-04-20
Duration: 179 days
Keywords: ['Trump', 'Ukraine', 'war', 'ceasefire', 'peace', 'negotiations', 'Zelensky']


In [3]:
all_articles = fetch_news_for_period(keywords, start_date, end_date)
print(f"\nTotal articles fetched: {len(all_articles)}")

Fetching news: 100%|██████████| 286/286 [07:33<00:00,  1.59s/req, articles=29716]


Total articles fetched: 29716


In [5]:
# Filter articles by relevance using cross-encoder
query_text = f"{event_info['title']} " + " ".join(keywords)
print(f"Query for relevance: {query_text}")
relevant_articles = filter_by_relevance(all_articles, query_text, threshold=-4.5)

Query for relevance: Trump ends Ukraine war in first 90 days? Trump Ukraine war ceasefire peace negotiations Zelensky
Scoring 29716 articles for relevance...


Batches: 100%|██████████| 929/929 [00:18<00:00, 51.46it/s]


Filtered to 1463 relevant articles (threshold=-4.5)


In [ ]:
# Save relevant articles to CSV
csv_filename = f"news_{MARKET_NAME}_{EVENT_ID}.csv"
save_news_to_csv(relevant_articles, csv_filename)

### Other markets

In [3]:
MARKETS_TO_PROCESS = [
    (903193, "kamala", "presidential-election-winner-2024-Kamala.json"),
    (903193, "trump", "presidential-election-winner-2024-Trump.json"),
    (21257, "israel", "israel-x-hamas-ceasefire-before-july-2025-79c05.json"),
    (12641, "tiktok", "tiktok-banned-in-the-us-before-may-2025-52cc0.json"),
]

for event_id, market_name, data_file in MARKETS_TO_PROCESS:
    print(f"\n{'='*60}")
    print(f"Processing: {market_name} (event {event_id})")
    print('='*60)
    
    data = load_market_data(filename=data_file)
    event_info = data['event']
    start_date = event_info['start_date']
    end_date = event_info['end_date']
    
    keywords = MARKET_KEYWORDS.get((event_id, market_name), [])
    print(f"Period: {start_date.date()} to {end_date.date()}")
    print(f"Keywords: {keywords}")
    
    articles = fetch_news_for_period(keywords, start_date, end_date)
    print(f"Total articles fetched: {len(articles)}")
    
    query_text = f"{event_info['title']} " + " ".join(keywords)
    relevant = filter_by_relevance(articles, query_text, threshold=-4.5)
    
    csv_filename = f"news_{market_name}_{event_id}.csv"
    save_news_to_csv(relevant, csv_filename)


Processing: kamala (event 903193)


Loading times: 100%|██████████| 439715/439715 [01:01<00:00, 7115.59point/s]


Period: 2024-01-04 to 2024-11-05
Keywords: ['election', 'Democrat', 'Harris', 'Kamala', 'vice president']


Fetching news: 100%|██████████| 484/484 [12:14<00:00,  1.52s/req, articles=57147]


Total articles fetched: 57147
Scoring 57147 articles for relevance...


...

Total articles fetched: 13563
Scoring 13563 articles for relevance...


Batches: 100%|██████████| 424/424 [00:08<00:00, 51.91it/s]


Filtered to 247 relevant articles (threshold=-4.5)


Saving to CSV: 100%|██████████| 247/247 [00:00<00:00, 211686.37row/s]

Saved 247 articles to /Users/imakarov/STUDY/ML-News-Market-Analysis/notebooks/../data/news_tiktok_12641.csv
